In [1]:
# note need to install transpile_benchy before running
# run 'pip install git+https://github.com/evmckinney9/transpile_benchy.git'
# also need to run 'pip install mqt.bench'
import qiskit

In [17]:
# from transpile_benchy.interface import QASMBench
# from transpile_benchy.interface import QiskitFunctionInterface
# from transpile_benchy.src.transpile_benchy.interfaces.abc_interface import SubmoduleInterface
from transpile_benchy.src.transpile_benchy.metrics.abc_metrics import MetricInterface

# from transpile_benchy.src.transpile_benchy.metrics.gate_counts import DepthMetric

# from transpile_benchy.src.transpile_benchy.metrics.gate_counts import DepthMetric
from transpile_benchy.src.transpile_benchy.interfaces.qasm_interface import QASMBench
from transpile_benchy.src.transpile_benchy.library import CircuitLibrary

# from transpile_benchy.src.transpile_benchy.interfaces.qasm_interface import QASMBench
# from qiskit.transpiler import CouplingMap
from typing import Iterator, List
from qiskit import QuantumCircuit

# circuits to test
from qiskit.circuit.library import QuantumVolume
from qiskit.circuit.library import PhaseEstimation
from qiskit.circuit.library.basis_change import QFT
from qiskit.circuit.library import LinearAmplitudeFunction
from qiskit.circuit.library import CDKMRippleCarryAdder


from supermarq.benchmarks.ghz import GHZ
from supermarq.benchmarks.hamiltonian_simulation import HamiltonianSimulation
from supermarq.benchmarks.qaoa_vanilla_proxy import QAOAVanillaProxy
from supermarq.converters import cirq_to_qiskit

qasmBench = QASMBench(size="small")
library = CircuitLibrary.from_txt("small_circuits.txt")


# monodromy import wasn't working so define total metric below
class TotalMetric(MetricInterface):
    """Calculate the total 2Q gate count of a circuit."""

    def __init__(self):
        """Initialize the metric."""
        super().__init__(name="Total Gates", pretty_name="Total 2Q Gates")

    def _construct_pass(self, circuit: QuantumCircuit) -> float:
        """Calculate the depth of a circuit."""
        # dumb way :)
        from qiskit.converters import circuit_to_dag

        dag = circuit_to_dag(circuit)
        return len(dag.two_qubit_ops())


# size = 28

# qv_bench = CircuitBench(size=size, circuitName="QuantumVolume")
# qft_bench = CircuitBench(size=size, circuitName="QFT")
# ripple_adder_bench = CircuitBench(size=size, circuitName="RippleAdder")
# phase_est_bench = CircuitBench(size=size, circuitName="PhaseEstimation")
# linear_amp_bench = CircuitBench(size=size, circuitName="LinearAmplitude")
# qaoa_vanilla_bench  = CircuitBench(size=size, circuitName="QAOA_Vanilla")
# tim_ham_bench = CircuitBench(size=size, circuitName="TIM_Hamiltonian")
# adder_bench = CircuitBench(size=size, circuitName="Adder")
# GHZ_bench = CircuitBench(size=size, circuitName="GHZ")

# updated way to call qiskit functions
# num_qubits = [size]
# qv_bench = QiskitFunctionInterface(function_type = QuantumVolume, num_qubits=num_qubits)
# qft_bench = QiskitFunctionInterface(function_type = QFT, num_qubits=num_qubits)
# phase_est_bench = QiskitFunctionInterface(function_type=PhaseEstimation, num_qubits=num_qubits)

# qaoa_vanilla_bench  = QiskitFunctionInterface(function_type = cirq_to_qiskit(QAOAVanillaProxy.circuit()), num_qubits=num_qubits)
# tim_ham_bench = QiskitFunctionInterface(function_type = cirq_to_qiskit(HamiltonianSimulation.circuit()), num_qubits=num_qubits)
# adder_bench = QiskitFunctionInterface(function_type = cirq_to_qiskit(QuantumCircuit().compose(CDKMRippleCarryAdder()
#             .decompose()
#             .decompose()
#             .decompose())), num_qubits=num_qubits)
# GHZ_bench = QiskitFunctionInterface(function_type = cirq_to_qiskit(GHZ().circuit()), num_qubits=num_qubits)


# from transpile_benchy.metrics import DepthMetric

# depth_metric = DepthMetric()
total_metric = TotalMetric()

In [23]:
from qiskit import transpile
from abc import abstractmethod

from qiskit.transpiler import PassManager

# from transpile_benchy.src.transpile_benchy.runner import AbstractRunner
from transpile_benchy.src.transpile_benchy.passmanagers.abc_runner import (
    CustomPassManager,
)
from transpile_benchy.src.transpile_benchy.passmanagers.qiskit_baseline import (
    QiskitBaseline,
)


class QiskitStage(PassManager):
    """QiskitStage uses Qiskit's built-in transpilation strategies."""

    default_basis = ["u", "cx", "id", "measure", "reset", "barrier"]
    CONFIGS = {
        1: {"optimization_level": 1, "basis_gates": default_basis},
        2: {"optimization_level": 2, "basis_gates": default_basis},
        3: {"optimization_level": 3, "basis_gates": default_basis},
    }

    def __init__(self, **transpiler_kwargs):
        """Initialize the QiskitStage."""
        self.transpiler_kwargs = transpiler_kwargs

    @classmethod
    def from_predefined_config(cls, optimization_level: int, **kwargs):
        """Create a pre-defined QiskitStage.

        Create a QiskitStage from a pre-defined transpiler configuration
        and merge it with provided kwargs.
        """
        config = cls.CONFIGS.get(optimization_level)
        if not config:
            raise ValueError(f"Invalid level. Choose from {list(cls.CONFIGS.keys())}.")
        config.update(kwargs)  # Merge predefined config with provided kwargs
        return cls(**config)

    def run(self, circuit):
        """Run the transpiler on the circuit."""
        return transpile(circuit, **self.transpiler_kwargs)


class QiskitTranspileRunner(CustomPassManager):
    """used to noop the pre-, main-, post- passes"""

    def __init__(self, coupling_map, name: str = None, circuit=None):
        super().__init__(name=name)
        self.coupling_map = coupling_map
        self.circuit = circuit

    def pre_process(self):
        pass

    def main_process(self, qc):
        self.circuit = transpile(
            qc,
            coupling_map=self.coupling_map,
            optimization_level=3,
            basis_gates=["cx", "u"],
        )
        return self.circuit

    def post_process(self):
        pass

    def stage_builder(self):
        """Build stages in a defined sequence."""

        def _builder():
            yield QiskitStage.from_predefined_config(
                optimization_level=self.optimization_level, **self.transpiler_kwargs
            )

        return _builder


# class corral(QiskitTranspileRunner):
#     def __init__(self, coupling_map, name:str=None):
#         super().__init__(name=name)
#         self.coupling_map = coupling_map
#     def run(self, qc):
#         return transpile(qc, coupling_map=self.coupling_map, optimization_level=3, basis_gates=["cx", "u"])

In [24]:
def snail_to_connectivity(snail_edge_list):
    # Convert snail edge list where nodes are snails and edges are qubits
    # To connectivity edge list where nodes are qubits and edges are coupling
    edge_list = []

    # qubits are coupled to a snail edge if they are both adjacent to a snail node
    for qubit, snail_edge in enumerate(snail_edge_list):
        for temp_qubit, temp_snail_edge in enumerate(snail_edge_list):
            if qubit != temp_qubit and (
                snail_edge[0] in temp_snail_edge or snail_edge[1] in temp_snail_edge
            ):
                edge_list.append((qubit, temp_qubit))
    return edge_list


# define corral
def corral_skip(num_snails=32, num_levels=2, level_1_skip=1, level_2_skip=1):
    """Returns edge list of a corral of size specified snails are nodes,
    edges are qubits."""

    snail_edge_list = []
    for snail0, snail1 in zip(range(num_snails), range(1, num_snails + 1)):
        for i in range(1, num_levels + 1):
            if i == 1:
                snail_edge_list.append(((snail0 + level_1_skip) % num_snails, snail0))
            elif i == 2:
                snail_edge_list.append((snail0, (snail0 + level_2_skip) % num_snails))
    return snail_edge_list


from qiskit.transpiler.coupling import CouplingMap
import retworkx as rx
import random
import logging

from src.clonk.backend_utils.mock_backends.Corral_backend_v1 import FakeCorral
import numpy as np


def pretty_print(edge_list, simple=True):
    if (
        simple
    ):  # simple only shows one connection if reverse node connection is there too
        temp = list(edge_list)
        temp2 = []
        for i, j in temp:
            if not (j, i) in temp2:
                temp2.append((i, j))
        edge_list = temp2

    x = CouplingMap(edge_list)

    # black magic errors when I modify the draw function directly in the CouplingMap file so Im just copying the code here to make it work
    import pydot
    from PIL import Image
    import io

    formatter2 = lambda edge: dict(dir="none", color="blue")

    pgraph = rx.PyGraph()

    graph_edges = x.get_edges()
    new_edges = []
    for e in graph_edges:
        e = list(e)
        e.append(1)
        new_edges.append(tuple(e))
    pgraph.add_nodes_from(list(range(64)))  # update so not specific to 64 qubit corrals
    pgraph.add_edges_from(new_edges)
    # update edge colors?
    graph_colors = rx.graph_greedy_color(pgraph)

    # for node in graph_colors:
    #     print(node)
    #     x.graph[node] = graph_colors[node] # z

    dot_str = x.graph.to_dot(
        edge_attr=formatter2, graph_attr={"size": "0", "color": "blue"}
    )

    dot = pydot.graph_from_dot_data(dot_str)[0]
    png = dot.create_png(prog="sfdp")
    pdf = dot.create_pdf(prog="sfdp")
    png = dot.create_png(prog="neato")
    return Image.open(io.BytesIO(png))

In [25]:
from qiskit.transpiler.coupling import CouplingMap
import math

input_corrals = {}
x_axis_values = []
size = 20
N = math.ceil(size / 2) + 1  # N is number of snails 2*N is number of qubits
first = 1
for i in range(
    1, N // 2 + 1, 1
):  # only need to do half of N connections since >N/2 makes repeat corrals
    name = f"corral_n{2*N}_({first}_{i})"
    x_axis_values.append(i)
    input_corrals[i - 1] = CouplingMap(
        snail_to_connectivity(
            corral_skip(num_snails=N, level_1_skip=first, level_2_skip=i)
        ),
        description=name,
    )


# display(pretty_print(snail_to_connectivity(corral_skip(num_snails = N, level_1_skip=first, level_2_skip=1))))
# display(pretty_print(snail_to_connectivity(corral_skip(num_snails = N, level_1_skip=first, level_2_skip=12))))
# display(pretty_print(snail_to_connectivity(corral_skip(num_snails = N, level_1_skip=first, level_2_skip=28))))
# display(pretty_print(snail_to_connectivity(corral_skip(num_snails = N, level_1_skip=first, level_2_skip=32))))
# display(pretty_print(snail_to_connectivity(corral_skip(num_snails = N, level_1_skip=first, level_2_skip=48))))

# ND = 16
# display(pretty_print(corral_skip(num_snails = ND, level_1_skip=first, level_2_skip=1) , simple =  False))
# display(pretty_print(corral_skip(num_snails = ND, level_1_skip=first, level_2_skip=2), simple =  False))
# display(pretty_print(corral_skip(num_snails = ND, level_1_skip=first, level_2_skip=3), simple =  False))
# display(pretty_print(corral_skip(num_snails = ND, level_1_skip=first, level_2_skip=4), simple =  False))
# display(pretty_print(corral_skip(num_snails = ND, level_1_skip=first, level_2_skip=6), simple =  False))
# display(pretty_print(corral_skip(num_snails = ND, level_1_skip=first, level_2_skip=7), simple =  False))
# display(pretty_print(corral_skip(num_snails = ND, level_1_skip=first, level_2_skip=8), simple =  False))

In [ ]:
from logging import Logger
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors


def plot(b=None, save=False):
    """Plot benchmark results.
    b is the bencmark instance, needed to overwrite plot func in class w/o changing source code
    """
    with plt.style.context("seaborn-darkgrid"):
        bar_width = 0.35
        transpiler_count = len(b.transpilers)

        # Define color palette, may need to add more colors in data exceeds current number of colors
        colors = [
            "#ff0000",
            "#ff8000",
            "#ffff00",
            "#80ff00",
            "#009900",
            "#00cccc",
            "#0000cc",
            "#b266ff",
            "#ff66b2",
            "#606060",
            "#1f77b4",
            "#ff7f0e",
            "#0000cc",
            "#ffcce5",
            "#B6196D",
            "#A800A8",
            "#5F69F7",
            "#5FBAF7",
            "#1EE7EE",
            "#1EEE78",
            "#ACF954",
            "#E8F954",
            "#D1A513",
            "#FF007f",
        ]

        # Loop over metrics
        for metric_name, results in b.results.items():
            labels = []
            plotlines = {}  # x y info for each corral

            fig, ax = plt.subplots()
            qubits = b.submodules[0].rangeQ
            # Create a line for each circuit
            for i, (circuit_name, circuit_results) in enumerate(results.items()):
                # Create a line for each qubit at different corral connection patterns
                for j, transpiler_result in enumerate(circuit_results):
                    if (
                        qubits[i] not in labels
                    ):  # transpiler result is str of corral name
                        labels.append(qubits[i])
                    # ax.scatter(qubits[i], circuit_results[transpiler_result], c = colors[j%len(colors)], label = transpiler_result,linewidth=2.0)
                    if qubits[i] in plotlines:
                        plotlines[qubits[i]][0].append(int(transpiler_result))
                        plotlines[qubits[i]][1].append(
                            circuit_results[transpiler_result]
                        )
                    else:
                        plotlines[qubits[i]] = [
                            [int(transpiler_result)],
                            [circuit_results[transpiler_result]],
                        ]
                        # each line for different qubit, x is corral, y is depth

            max_height = 0
            max_x = 0
            plt.figure(figsize=(15, 10))
            for i, c in enumerate(plotlines):
                plt.plot(plotlines[c][0], plotlines[c][1], marker="o", color=colors[i])
                if max(plotlines[c][1]) > max_height:
                    max_height = max(plotlines[c][1])
                if max(plotlines[c][0]) > max_x:
                    max_x = max(plotlines[c][0])

            # Add labels, title, etc
            plt.xlabel("Skip connections (1,x)")
            plt.ylabel(metric_name)
            # subtitle Best of N={self.num_runs} runs
            plt.title(
                f"Transpiler {metric_name} Comparison,\
                        Best of N={b.num_runs} runs"
            )

            max_fontsize = 10
            min_fontsize = 4
            font_size = max(min(max_fontsize, 800 // len(results)), min_fontsize)

            ax.set(
                xlim=(0, max_x + 2),
                xticks=np.arange(1, max_x + 2, 5),
                ylim=(0, max_height + 10),
                yticks=np.arange(1, max_height + 10, 500),
            )
            plt.legend(labels)  # number of qubits in circuit

            # Show the plot
            plt.show()

            if save:
                plt.savefig(f"transpile_benchy_{metric_name}.svg", dpi=300)

In [ ]:
from logging import Logger
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors


def plotLines(b=None, save=False):
    with plt.style.context("seaborn-darkgrid"):
        bar_width = 0.35
        transpiler_count = len(b.transpilers)

        # Define color palette, may need to add more colors in data exceeds current number of colors
        colors = [
            "#ff0000",
            "#ff8000",
            "#ffff00",
            "#80ff00",
            "#009900",
            "#00cccc",
            "#0000cc",
            "#b266ff",
            "#ff66b2",
            "#606060",
            "#1f77b4",
            "#ff7f0e",
            "#0000cc",
            "#ffcce5",
            "#B6196D",
            "#A800A8",
            "#5F69F7",
            "#5FBAF7",
            "#1EE7EE",
            "#1EEE78",
            "#ACF954",
            "#E8F954",
            "#D1A513",
            "#FF007f",
        ]

        circuits = [
            "qv",
            "qft"
            # 'phase_estimation'
            # 'qaoa_vanilla'
            # 'tim',
            # 'adder',
            # 'GHZ'
        ]

        # Loop over metrics
        for metric_name in b.results.results.keys():
            results = b.results.results[metric_name]
            transpiler_count = len(b.transpilers)
            labels = []
            plotlines = {}  # x y info for each corral

            fig, ax = plt.subplots()
            qubits = b.submodules[0].rangeQ
            # Create a line for each circuit
            for i, (circuit_name, circuit_results) in enumerate(results.items()):
                # print(f'i: {i}, circ name: {circuit_name}')
                # Create a line for each qubit at different corral connection patterns
                for j, transpiler in enumerate(b.transpilers):
                    result_metrics = b.results.get_metrics(
                        metric_name, circuit_name, transpiler.name
                    )
                    # print(f'j: {j}, transpiler res: {transpiler_result}')
                    label = circuits[i]
                    if label not in labels:  # transpiler result is str of corral name
                        labels.append(label)
                    # ax.scatter(qubits[i], circuit_results[transpiler_result], c = colors[j%len(colors)], label = transpiler_result,linewidth=2.0)
                    if label in plotlines:
                        plotlines[label][0].append(x_axis_values[j])
                        plotlines[label][1].append(result_metrics.average)
                    else:
                        plotlines[label] = [
                            [x_axis_values[j]],
                            [result_metrics.average],
                        ]
                        # each line for different qubit, x is corral, y is depth

            max_height = 0
            max_x = 0
            plt.figure(figsize=(15, 10))
            for i, c in enumerate(plotlines):
                plt.plot(plotlines[c][0], plotlines[c][1], marker="o", color=colors[i])
                if max(plotlines[c][1]) > max_height:
                    max_height = max(plotlines[c][1])
                if max(plotlines[c][0]) > max_x:
                    max_x = max(plotlines[c][0])

            # Add labels, title, etc
            plt.xlabel("Skip connections (1,x)")
            plt.ylabel(metric_name)
            # subtitle Best of N={self.num_runs} runs
            plt.title(
                f"Transpiler {metric_name} Comparison,\
                        Best of N={b.num_runs} runs"
            )

            max_fontsize = 10
            min_fontsize = 4
            font_size = max(min(max_fontsize, 800 // len(results)), min_fontsize)

            ax.set(
                xlim=(0, max_x + 2),
                xticks=np.arange(1, max_x + 2, 5),
                ylim=(0, max_height + 10),
                yticks=np.arange(1, max_height + 10, 500),
            )
            plt.legend(labels)  # number of qubits in circuit

            # Show the plot
            plt.show()

In [ ]:
def _loadData(label):
    """load data as dict wit key as freq"""
    logging.info("loading data")
    with h5py.File("data.hdf5", "r+") as f:
        group = f.require_group("corral_skip_patterns")
        data_list = group.require_dataset(label, shape=(1, 2), dtype="float32")
        return data_list[()]


# label is circuit name, returns dict w/ key corral_pattern_qubits, value is [depth, gatecount]
def _saveData(self, label, data_list):
    """save data using h5py"""
    logging.info("saving data")
    # save for cavity drive freq
    with h5py.File("data.hdf5", "w") as f:
        group = f.require_group("corral_skip_patterns")
        group.create_dataset(name=label, data=data_list)

In [ ]:
plotLines(benchmark, save=True)

In [26]:
from transpile_benchy.src.transpile_benchy.benchmark import Benchmark
from transpile_benchy.src.transpile_benchy.passmanagers.qiskit_baseline import (
    QiskitBaseline,
)

# Create a specific logger for transpile_benchy and set its level to DEBUG
transpile_benchy_logger = logging.getLogger("transpile_benchy")
# transpile_benchy_logger.setLevel(logging.DEBUG)
transpile_benchy_logger.setLevel(logging.WARNING)

# Silence the qiskit logger by setting its level to WARNING or higher
qiskit_logger = logging.getLogger("qiskit")
qiskit_logger.setLevel(logging.WARNING)
transpilersCorral = []
for c in input_corrals:
    print(input_corrals[c].description)
    transpilersCorral.append(
        QiskitBaseline(
            optimization_level=3,
            CouplingMap=input_corrals[c],
            name=input_corrals[c].description,
        )
    )  # name is number of skipped qubits in connections connection would be (1,c)

benchmark = Benchmark(
    transpilers=transpilersCorral,
    circuit_library=library,
    metrics=[total_metric],
    logger=transpile_benchy_logger,
    num_runs=3,
)
benchmark.run()
benchmark.plot(save=True)
# plot(b = benchmark, save=True)

corral_n22_(1_1)
corral_n22_(1_2)
corral_n22_(1_3)
corral_n22_(1_4)
corral_n22_(1_5)


AttributeError: 'NoneType' object has no attribute 'name'